In [1]:
# Install necessary libraries (run only if not already installed)
!pip install google-cloud-storage google-cloud-documentai google-cloud-aiplatform vertexai

  Using cached google_cloud_documentai-3.5.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached google_cloud_core-2.4.3-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_cloud_storage-2.19.0-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached google_cloud_bigquery-3.35.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached google_cloud_resource_manager-1.14.2-py3-none-any.whl.metadata (9.6 kB)
  Using cached docstring_parser-0.17.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached grpc_google_iam_v1-0.14.2-py3-none-any.whl.metadata (9.1 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
Using cached google_cloud_documentai-3.5.0-py3-none-any.whl (299 kB)
Using cached google_cloud_storage-2.19.0-py2.py3-none-any.whl (131 kB)
   ----------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.3.68 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.
langchain-google-genai 2.1.6 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [2]:
# Imports
from google.cloud import storage, documentai_v1 as documentai
from vertexai.preview import rag
from vertexai.generative_models import GenerativeModel, Tool
import vertexai
import os
from google.oauth2 import service_account

# Path to your service account key JSON file
key_path = r"C:\Users\jagad\Agentic AI-planner and content generator\agentic-ai-project-466814-869df6973c65.json"

# Create credentials object
credentials = service_account.Credentials.from_service_account_file(key_path)

from google.cloud import storage
PROJECT_ID = "agentic-ai-project-466814"
client = storage.Client(project=PROJECT_ID, credentials=credentials)

In [3]:
# Set your project and location
PROJECT_ID = "agentic-ai-project-466814"
LOCATION = "us-central1"  # Use supported Vertex AI RAG Engine region
vertexai.init(project=PROJECT_ID, location=LOCATION)



try:
    client = storage.Client(credentials=credentials, project=PROJECT_ID)
    bucket_name = "studyplanandcontent"
    bucket = client.bucket(bucket_name)
except Exception as e:
    print(f"[GCS ERROR] Client or bucket initialization failed: {e}")
    raise

local_folder = "C:/Users/jagad/Downloads/iesc1dd"
file_path = "C:/Users/jagad/Downloads/iesc1dd/iesc103.pdf"
file_name = os.path.basename(file_path)
try:
    for file_name in os.listdir(local_folder):
        blob = bucket.blob(file_name)
        blob.upload_from_filename(os.path.join(local_folder, file_name))
        print(f"[UPLOAD SUCCESS] Uploaded {file_name}")
except Exception as e:
    print(f"[GCS UPLOAD ERROR] Uploading {file_name} failed: {e}")
    raise

In [4]:
from google.cloud import documentai_v1 as documentai
from google.cloud.documentai_v1 import types
processor_id = "6969be315242bcf5"
region = "US"

gcs_input_uri = "gs://studyplanandcontent/iesc106.pdf"
gcs_output_uri = "gs://studyplanandcontent/output/"  # Output prefix folder

# The full resource name of the processor
name = f"projects/agentic-ai-project-466814/locations/us/processors/6969be315242bcf5"

client = documentai.DocumentProcessorServiceClient(credentials=credentials)
# Configure input documents (GCS)
gcs_documents = types.GcsDocuments(
    documents=[
        types.GcsDocument(
            gcs_uri=gcs_input_uri, mime_type="application/pdf"
        )
    ]
)

input_config = types.BatchDocumentsInputConfig(gcs_documents=gcs_documents)

# Configure output bucket
output_config = types.DocumentOutputConfig(
    gcs_output_config=types.DocumentOutputConfig.GcsOutputConfig(
        gcs_uri=gcs_output_uri
    )
)

# Construct request
request = types.BatchProcessRequest(
    name=name,
    input_documents=input_config,
    document_output_config=output_config
)

# Process document asynchronously
operation = client.batch_process_documents(request=request)
print("Waiting for operation to complete...")
operation.result(timeout=300)

print("[DOCUMENT AI SUCCESS] Batch processing complete. Check your GCS output folder.")

Waiting for operation to complete...
[DOCUMENT AI SUCCESS] Batch processing complete. Check your GCS output folder.


In [44]:
!pip install vertexai --upgrade

In [48]:
!pip install --quiet google-cloud-storage vertexai numpy

In [65]:
import os
import json
import numpy as np
from io import BytesIO
from google.cloud import storage
from vertexai import init
from vertexai.preview.language_models import TextEmbeddingModel
from vertexai.preview import rag

# Configuration
PROJECT_ID = "agentic-ai-project-466814"
LOCATION = "us-central1" 
BUCKET_NAME = "studyplanandcontent"
DOC_JSON_PATH = "output/17996961397770261561/0/iesc106-0.json"
GCS_SAVE_PATH = "embeddingsoutput/iesc106_vector.npy"

# 🟢 Step 1: Init Vertex AI
print("⏳ Initializing Vertex AI...")
init(project=PROJECT_ID, location=LOCATION)
print("✅ Vertex AI Initialized")

# 📥 Step 2: Read Document AI Output JSON from GCS
print("⏳ Fetching Document AI output JSON...")
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob(DOC_JSON_PATH)
docai_json = json.loads(blob.download_as_text())
print("✅ JSON Loaded")

# 📝 Step 3: Extract Text
document_text = docai_json.get("text", "")
print(f"📄 Text Extracted: {len(document_text)} characters")

# 🤖 Step 4: Generate Embedding
print("⏳ Generating embedding using Vertex AI model...")
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")
embedding_response = embedding_model.get_embeddings([document_text])
embedding_vector = embedding_response[0].values
print("✅ Embedding generated")

# ☁️ Step 5: Upload Embedding Directly to GCS (No local file)
print("⏳ Uploading embedding to GCS...")
buffer = BytesIO()
np.save(buffer, np.array(embedding_vector))
buffer.seek(0)

embedding_blob = bucket.blob(GCS_SAVE_PATH)
embedding_blob.upload_from_file(buffer, content_type="application/octet-stream")
print(f"✅ Embedding uploaded to: gs://{BUCKET_NAME}/{GCS_SAVE_PATH}")



⏳ Initializing Vertex AI...
✅ Vertex AI Initialized
⏳ Fetching Document AI output JSON...
✅ JSON Loaded
📄 Text Extracted: 24018 characters
⏳ Generating embedding using Vertex AI model...
✅ Embedding generated
⏳ Uploading embedding to GCS...
✅ Embedding uploaded to: gs://studyplanandcontent/embeddingsoutput/iesc106_vector.npy


In [66]:
from io import BytesIO

blob = bucket.blob(GCS_SAVE_PATH)
buffer = BytesIO()
blob.download_to_file(buffer)
buffer.seek(0)
doc_embedding_vector = np.load(buffer)
print(f"✅ Loaded document embedding: {doc_embedding_vector.shape}")

✅ Loaded document embedding: (768,)


In [73]:
user_question = "What is a nervous tissue?" 
question_embedding_response = embedding_model.get_embeddings([user_question])
question_vector = np.array(question_embedding_response[0].values)

In [74]:
from numpy.linalg import norm

def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

similarity = cosine_similarity(question_vector, doc_embedding_vector)
print(f"📊 Cosine similarity: {similarity:.4f}")


📊 Cosine similarity: 0.6144


In [79]:
from vertexai.preview.generative_models import GenerativeModel
from vertexai.preview.generative_models import GenerationConfig
from vertexai import init

# Make sure you're initializing the correct project and region
init(project="agentic-ai-project-466814", location="us-central1")

# Now create the model
gemini = GenerativeModel("gemini-2.5-flash")  # Works now because init() handles the context

prompt = f"""
You're a helpful tutor. Here's some science material:

--- Start of Content ---
{document_text[:3000]}
--- End of Content ---

Question: {user_question}
Answer in a simple, student-friendly way.
"""

response = gemini.generate_content(prompt)
print("💬 Gemini Answer:")
print(response.text)


💬 Gemini Answer:
Based on the text, here's a simple explanation:

Nervous tissue is a special type of tissue in your body that's mainly made up of cells called **nerve cells** (or neurons).

Its main job, as the text mentions, is to **carry messages**. Think of it like the body's super-fast communication system! These nerve cells work together to send electrical signals and information between your brain, spinal cord, and all the different parts of your body. This allows you to feel things, move, think, and react.


In [80]:
import math

# Set your desired chunk size (e.g., 500 tokens or ~500-1000 characters)
chunk_size = 1000
document_chunks = []
for i in range(0, len(document_text), chunk_size):
    document_chunks.append(document_text[i:i+chunk_size])

print(f"🔖 Document split into {len(document_chunks)} chunks.")


🔖 Document split into 25 chunks.


In [81]:
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")

chunk_embeddings = []
chunk_metadata = []

for idx, chunk in enumerate(document_chunks):
    resp = embedding_model.get_embeddings([chunk])
    chunk_embeddings.append(resp[0].values)
    chunk_metadata.append({
        "chunk_id": idx,
        "start": idx * chunk_size,
        "end": idx * chunk_size + len(chunk),
        "text": chunk
    })

chunk_embeddings_array = np.array(chunk_embeddings)
# Save embedding vectors
emb_buffer = BytesIO()
np.save(emb_buffer, chunk_embeddings_array)
emb_buffer.seek(0)
bucket.blob("embeddingsoutput/iesc106_vector_chunks.npy").upload_from_file(emb_buffer, content_type="application/octet-stream")

# Save metadata (chunk text, positions)
import pandas as pd
meta_df = pd.DataFrame(chunk_metadata)
csv_buffer = BytesIO()
meta_df.to_csv(csv_buffer, index=False)
csv_buffer.seek(0)
bucket.blob("embeddingsoutput/iesc106_chunk_metadata.csv").upload_from_file(csv_buffer, content_type="text/csv")
print("✅ Chunked embeddings and metadata saved to GCS.")


✅ Chunked embeddings and metadata saved to GCS.


In [82]:
# Load embeddings and metadata from GCS (reuse your existing GCS code)
emb_blob = bucket.blob("embeddingsoutput/iesc106_vector_chunks.npy")
emb_buf = BytesIO()
emb_blob.download_to_file(emb_buf)
emb_buf.seek(0)
chunk_embeds = np.load(emb_buf)

meta_blob = bucket.blob("embeddingsoutput/iesc106_chunk_metadata.csv")
csv_buf = BytesIO()
meta_blob.download_to_file(csv_buf)
csv_buf.seek(0)
meta_df = pd.read_csv(csv_buf)

# Embed the user query
user_question = "What is a nervous tissue?"
query_vec = embedding_model.get_embeddings([user_question])[0].values

# Compute cosine similarity for each chunk
from numpy.linalg import norm
def cosine_similarity(a, b): return np.dot(a, b) / (norm(a) * norm(b))
similarities = [cosine_similarity(query_vec, vec) for vec in chunk_embeds]
best_idx = int(np.argmax(similarities))
best_chunk = meta_df.iloc[best_idx]

print("📢 Quoted Chunk:\n", best_chunk['text'])


📢 Quoted Chunk:
 6
SCIENCE
TISSUES
then transmitting the stimulus very rapidly
from one place to another within the body. The
brain, spinal cord and nerves are all composed
of the nervous tissue. The cells of this tissue
are called nerve cells or neurons. A neuron
consists of a cell body with a nucleus and
cytoplasm, from which long thin hair-like
parts arise (Fig. 6.12). Usually each neuron
has a single long part (process) in the form of
a fibre, called the axon, and many short,
Nucleus
Cell body
Dendrite
Axon
Nerve ending
Fig. 6.12: Neuron-a unit of nervous tissue
branched parts (processes) called dendrites.
An individual nerve cell may be up to a metre
long. Many nerve fibres bound together by
connective tissue make up a nerve.
The signal that passes along the nerve fibre
is called a nerve impulse. The nerve impulse
from the nerve endings in transmitted to the
dendrites of the next nerve cell. Nerve
impulses allow us to move our muscles when
we want to. The functional combination of

In [83]:
prompt = f'''
Here's a quote from the source document:

"{best_chunk['text']}"

Explain the above in simple terms for a student.
'''

response = gemini.generate_content(prompt)
print("💬 Explanation:\n", response.text)
print(f"[Chunk {best_idx}, chars {best_chunk['start']}-{best_chunk['end']}]")
print("Quoted Text:", best_chunk['text'])
print("\nExplanation:", response.text)


💬 Explanation:
 Okay, imagine your body has its own super-fast communication system – that's what **nervous tissue** is all about!

Here's a breakdown in simple terms:

1.  **What is it?**
    *   It's a special type of tissue that helps your body send messages incredibly quickly from one place to another.

2.  **Where do you find it?**
    *   You find this special tissue in your **brain**, your **spinal cord** (which runs down your back), and in all the **nerves** spread throughout your body.

3.  **What are its building blocks?**
    *   The tiny building blocks of this communication system are special cells called **nerve cells** or **neurons**.

4.  **What does a neuron look like?**
    *   Think of a neuron like a tiny tree or a strange-looking star:
        *   It has a main "body" (the **cell body**) where the "brain" of the cell (the nucleus) is.
        *   It has one very long "branch" or "tail" called an **axon**. This is like the main cable that carries the message *away* 

In [38]:
import os
import json
import numpy as np
from io import BytesIO
from google.cloud import storage
from vertexai import init
from vertexai.preview.language_models import TextEmbeddingModel
import pandas as pd

# Configuration
PROJECT_ID = "agentic-ai-project-466814"
LOCATION = "us-central1"
BUCKET_NAME = "studyplanandcontent"
DOC_JSON_PATH = "output/17996961397770261561/0/iesc106-0.json"
GCS_SAVE_PATH_PREFIX = "embeddingsoutput/iesc106_vector_chunk"
GCS_META_PATH = "embeddingsoutput/iesc106_chunk_metadata.csv"

# Initialize Vertex AI
init(project=PROJECT_ID, location=LOCATION)

# Fetch Document AI output JSON from GCS
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob(DOC_JSON_PATH)
docai_json = json.loads(blob.download_as_text())

# Chunk full document text
document_text = docai_json.get("text", "")
chunk_size = 700
chunks = [document_text[i:i+chunk_size] for i in range(0, len(document_text), chunk_size)]

# Load embedding model
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")

# Metadata for saving (to help link embeddings to text later)
meta = []

# Generate embeddings, upload each chunk
for idx, chunk_text in enumerate(chunks):
    if not chunk_text.strip():
        continue
    embedding_response = embedding_model.get_embeddings([chunk_text])
    embedding_vector = embedding_response[0].values

    # Save embedding to bytes buffer and upload
    buffer = BytesIO()
    np.save(buffer, np.array(embedding_vector))
    buffer.seek(0)
    chunk_save_path = f"{GCS_SAVE_PATH_PREFIX}_{idx}.npy"
    embedding_blob = bucket.blob(chunk_save_path)
    embedding_blob.upload_from_file(buffer, content_type="application/octet-stream")

    # Collect metadata info (add whatever extra fields needed)
    meta.append({
        "chunk_idx": idx,
        "start": idx * chunk_size,
        "end": min((idx+1)*chunk_size, len(document_text)),
        "text": chunk_text,
        "embedding_path": chunk_save_path
    })

# Save metadata CSV to GCS
meta_df = pd.DataFrame(meta)
csv_buf = BytesIO()
meta_df.to_csv(csv_buf, index=False)
csv_buf.seek(0)
meta_blob = bucket.blob(GCS_META_PATH)
meta_blob.upload_from_file(csv_buf, content_type="text/csv")
print(f"Saved metadata for {len(meta_df)} chunks.")


Saved metadata for 35 chunks.


In [40]:
import numpy as np
import pandas as pd
from io import BytesIO
from google.cloud import storage
from vertexai.preview.language_models import TextEmbeddingModel
from vertexai.preview.generative_models import GenerativeModel

PROJECT_ID = "agentic-ai-project-466814"
BUCKET_NAME = "studyplanandcontent"
GCS_SAVE_PATH_PREFIX = "embeddingsoutput/iesc106_vector_chunk"
GCS_META_PATH = "embeddingsoutput/iesc106_chunk_metadata.csv"

# Set up GCS
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)

# Load metadata (to know chunk count and file names)
meta_blob = bucket.blob(GCS_META_PATH)
csv_buf = BytesIO()
meta_blob.download_to_file(csv_buf)
csv_buf.seek(0)
meta_df = pd.read_csv(csv_buf)

# Load all embeddings based on metadata file paths
embeddings = []
for idx, row in meta_df.iterrows():
    emb_blob = bucket.blob(row['embedding_path'])
    emb_buf = BytesIO()
    emb_blob.download_to_file(emb_buf)
    emb_buf.seek(0)
    vector = np.load(emb_buf)
    embeddings.append(vector)
embeddings = np.array(embeddings)  # Shape: (num_chunks, embedding_dim)

# Get user question embedding
user_question = "What is a nervous tissue?"
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")
query_vec = embedding_model.get_embeddings([user_question])[0].values

# Cosine similarity
from numpy.linalg import norm

def cosine_similarity(a, b):
    return np.dot(a, b)/(norm(a)*norm(b))

similarities = [cosine_similarity(query_vec, emb) for emb in embeddings]
best_idx = int(np.argmax(similarities))
best_chunk = meta_df.iloc[best_idx]

# Assuming meta_df and best_idx are already defined and loaded
best_chunk = meta_df.iloc[best_idx]  # best_idx=31 in your case

# Show the full text of the best chunk
print("Full chunk text:")
print(best_chunk['text'])

# Prepare the prompt and call Gemini
from vertexai.preview.generative_models import GenerativeModel
gemini = GenerativeModel("gemini-2.5-pro")
prompt = f'''
Here's a quote from the source document:

"{best_chunk['text']}"

Explain the above in simple terms for a student.
'''
response = gemini.generate_content(prompt)
print("Explanation generated by Gemini:")
print(response.text)


C:\Users\jagad\anaconda3\envs\testingopenai\Lib\site-packages\vertexai\_model_garden\_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Full chunk text:
ecide.
Such muscles are called voluntary muscles
Features
Shape
Number of nuclei
Position of nuclei
Striated
Smooth Cardiac
6.3.4 NERVOUS TISSUE
All cells possess the ability to respond to
stimuli. However, cells of the nervous tissue
are highly specialised for being stimulated and
68
Reprint 2025-26
SCIENCE
TISSUES
then transmitting the stimulus very rapidly
from one place to another within the body. The
brain, spinal cord and nerves are all composed
of the nervous tissue. The cells of this tissue
are called nerve cells or neurons. A neuron
consists of a cell body with a nucleus and
cytoplasm, from which long thin hair-like
parts arise (Fig. 6.12). Usually each neuron
has a single long part


C:\Users\jagad\anaconda3\envs\testingopenai\Lib\site-packages\vertexai\generative_models\_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Explanation generated by Gemini:
Of course! Here is an explanation of that text in simple terms for a student.

***

### Your Body's Super-Fast Messaging System: Nervous Tissue

Imagine your body is a giant team, and your brain is the coach. How does the coach instantly tell the players (your hands, feet, etc.) what to do? It uses a super-fast messaging system called the **nervous tissue**.

Here’s what the text explains:

**1. A Very Special Job**
*   Almost all cells in your body can react to things around them (like heat or touch).
*   But the cells in your nervous tissue are **specialists**. Their main job is to receive a message (a "stimulus") and pass it along *incredibly fast* from one part of your body to another.

**2. Where Do You Find It?**
*   This special tissue makes up your body's main communication network:
    *   **The Brain:** The main control center.
    *   **The Spinal Cord:** The major highway for messages going up and down your body.
    *   **The Nerves:** The 

In [41]:
def auto_chunk_text_by_lines(text, max_chunk_size=700, min_line_length=30):
    """
    Auto-detect paragraphs by grouping lines.

    Splits text by newlines, filters out very short lines (likely headers or noise),
    then groups lines until chunk size limit is reached.
    """

    lines = [line.strip() for line in text.split('\n') if line.strip()]
    chunks = []
    current_chunk = ""

    for line in lines:
        # Skip very short lines that may be separators or noise
        if len(line) < min_line_length:
            continue

        if len(current_chunk) + len(line) + 1 > max_chunk_size:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = line
        else:
            if current_chunk:
                current_chunk += "\n" + line
            else:
                current_chunk = line

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

# Example Usage
document_text = """Your long text input here, 
with some line breaks but no consistent double newline paragraph markers"""

chunks = auto_chunk_text_by_lines(document_text)
print(f"Total chunks created: {len(chunks)}")
for idx, chunk in enumerate(chunks):
    print(f"\n--- Chunk {idx} ---\n{chunk[:300]}...")  # print chunk preview


Total chunks created: 1

--- Chunk 0 ---
with some line breaks but no consistent double newline paragraph markers...


In [42]:
import json
import numpy as np
import pandas as pd
from io import BytesIO
from google.cloud import storage
from vertexai import init
from vertexai.preview.language_models import TextEmbeddingModel

# Configuration
PROJECT_ID = "agentic-ai-project-466814"
LOCATION = "us-central1"
BUCKET_NAME = "studyplanandcontent"
DOC_JSON_PATH = "output/17996961397770261561/0/iesc106-0.json"
GCS_EMBEDDING_PATH_PREFIX = "embeddingsoutput/iesc106_vector_chunk"
GCS_META_PATH = "embeddingsoutput/iesc106_chunk_metadata.csv"

print("[Init] Initializing Vertex AI...")
init(project=PROJECT_ID, location=LOCATION)
print("[Init] Vertex AI Initialized")

print("[GCS] Initializing Google Cloud Storage client...")
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)
print(f"[GCS] Bucket '{BUCKET_NAME}' connected")

# Download document JSON
print(f"[Fetch] Downloading document JSON from gs://{BUCKET_NAME}/{DOC_JSON_PATH} ...")
blob = bucket.blob(DOC_JSON_PATH)
docai_json = json.loads(blob.download_as_text())
print("[Fetch] Document JSON downloaded and parsed")

document_text = docai_json.get("text", "")
print(f"[Text] Total document length: {len(document_text)} characters")

def auto_chunk_text_by_lines(text, max_chunk_size=700, min_line_length=30):
    """
    Auto-detect paragraphs by grouping lines.
    Splits text by newlines, filters out short/noise lines, groups until near max_chunk_size.
    """
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    chunks = []
    current_chunk = ""

    for line in lines:
        # Skip very short lines (likely headers/noise)
        if len(line) < min_line_length:
            continue

        if len(current_chunk) + len(line) + 1 > max_chunk_size:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = line
        else:
            current_chunk = current_chunk + "\n" + line if current_chunk else line

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

print("[Chunk] Creating chunks of the document automatically...")
chunks = auto_chunk_text_by_lines(document_text, max_chunk_size=700)
print(f"[Chunk] Created {len(chunks)} chunks")

print("[Embed] Loading embedding model 'text-embedding-005'...")
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")
print("[Embed] Embedding model ready")

meta = []

for idx, chunk_text in enumerate(chunks):
    print(f"[Embed] Generating embedding for chunk {idx} (length={len(chunk_text)})...")
    embedding_response = embedding_model.get_embeddings([chunk_text])
    embedding_vector = embedding_response[0].values
    print(f"[Embed] Embedding vector size: {len(embedding_vector)}")

    buffer = BytesIO()
    np.save(buffer, np.array(embedding_vector))
    buffer.seek(0)

    chunk_save_path = f"{GCS_EMBEDDING_PATH_PREFIX}_{idx}.npy"
    embedding_blob = bucket.blob(chunk_save_path)
    embedding_blob.upload_from_file(buffer, content_type="application/octet-stream")
    print(f"[Upload] Uploaded chunk {idx} embedding to gs://{BUCKET_NAME}/{chunk_save_path}")

    meta.append({
        "chunk_idx": idx,
        "start": None,  # Optional: you can calculate character indices if needed
        "end": None,
        "text": chunk_text,
        "embedding_path": chunk_save_path
    })

print(f"[Meta] Prepared metadata for {len(meta)} chunks")

meta_df = pd.DataFrame(meta)
csv_buf = BytesIO()
meta_df.to_csv(csv_buf, index=False)
csv_buf.seek(0)

meta_blob = bucket.blob(GCS_META_PATH)
meta_blob.upload_from_file(csv_buf, content_type="text/csv")
print(f"[Meta] Uploaded metadata CSV to gs://{BUCKET_NAME}/{GCS_META_PATH}")

print("[Done] Step 1 complete: All chunks embedded and saved.")


[Init] Initializing Vertex AI...
[Init] Vertex AI Initialized
[GCS] Initializing Google Cloud Storage client...
[GCS] Bucket 'studyplanandcontent' connected
[Fetch] Downloading document JSON from gs://studyplanandcontent/output/17996961397770261561/0/iesc106-0.json ...
[Fetch] Document JSON downloaded and parsed
[Text] Total document length: 24018 characters
[Chunk] Creating chunks of the document automatically...
[Chunk] Created 31 chunks
[Embed] Loading embedding model 'text-embedding-005'...
[Embed] Embedding model ready
[Embed] Generating embedding for chunk 0 (length=688)...
[Embed] Embedding vector size: 768
[Upload] Uploaded chunk 0 embedding to gs://studyplanandcontent/embeddingsoutput/iesc106_vector_chunk_0.npy
[Embed] Generating embedding for chunk 1 (length=658)...
[Embed] Embedding vector size: 768
[Upload] Uploaded chunk 1 embedding to gs://studyplanandcontent/embeddingsoutput/iesc106_vector_chunk_1.npy
[Embed] Generating embedding for chunk 2 (length=656)...
[Embed] Embed

In [43]:
import numpy as np
import pandas as pd
from io import BytesIO
from google.cloud import storage
from vertexai.preview.language_models import TextEmbeddingModel
from vertexai.preview.generative_models import GenerativeModel
from numpy.linalg import norm

PROJECT_ID = "agentic-ai-project-466814"
BUCKET_NAME = "studyplanandcontent"
GCS_META_PATH = "embeddingsoutput/iesc106_chunk_metadata.csv"

print("[Init] Initializing Google Cloud Storage client...")
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)
print(f"[Init] Connected to bucket '{BUCKET_NAME}'")

print(f"[Load] Downloading metadata CSV from gs://{BUCKET_NAME}/{GCS_META_PATH} ...")
meta_blob = bucket.blob(GCS_META_PATH)
csv_buf = BytesIO()
meta_blob.download_to_file(csv_buf)
csv_buf.seek(0)

meta_df = pd.read_csv(csv_buf)
num_chunks = len(meta_df)
print(f"[Load] Loaded metadata for {num_chunks} chunks")

print("[Load] Downloading all chunk embeddings...")
embeddings = []
for idx, row in meta_df.iterrows():
    emb_blob = bucket.blob(row['embedding_path'])
    emb_buf = BytesIO()
    emb_blob.download_to_file(emb_buf)
    emb_buf.seek(0)
    vector = np.load(emb_buf)
    embeddings.append(vector)
    print(f"  [Load] Chunk {idx} embedding loaded, shape={vector.shape}")

embeddings = np.array(embeddings)
print(f"[Load] All embeddings loaded with shape {embeddings.shape}")

user_question = "What is a nervous tissue?"
print(f"[Query] Embedding user question: '{user_question}'")
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")
query_vec = embedding_model.get_embeddings([user_question])[0].values
print(f"[Query] Query embedding vector size: {len(query_vec)}")

def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

print("[Compare] Computing cosine similarities...")
similarities = []
for idx, chunk_vec in enumerate(embeddings):
    sim = cosine_similarity(query_vec, chunk_vec)
    similarities.append(sim)
    print(f"  [Compare] Chunk {idx}: similarity={sim:.4f}")

best_idx = int(np.argmax(similarities))
best_similarity = similarities[best_idx]
best_chunk = meta_df.iloc[best_idx]

print(f"\n[Result] Best chunk index: {best_idx} with similarity: {best_similarity:.4f}")
print(f"[Result] Chunk text preview:\n{best_chunk['text'][:500]}...\n")

print("[Gemini] Initializing Gemini model...")
gemini = GenerativeModel("gemini-2.5-pro")
prompt = f'''
Here's a quote from the source document:

"{best_chunk["text"]}"

Explain the above in simple terms for a student.
'''
print("[Gemini] Sending prompt to Gemini...")
response = gemini.generate_content(prompt)
print(f"[Gemini] Response received (length: {len(response.text)})")

print("\n=== Explanation ===")
print(response.text)
print("="*80)


[Init] Initializing Google Cloud Storage client...
[Init] Connected to bucket 'studyplanandcontent'
[Load] Downloading metadata CSV from gs://studyplanandcontent/embeddingsoutput/iesc106_chunk_metadata.csv ...
[Load] Loaded metadata for 31 chunks
[Load] Downloading all chunk embeddings...
  [Load] Chunk 0 embedding loaded, shape=(768,)
  [Load] Chunk 1 embedding loaded, shape=(768,)
  [Load] Chunk 2 embedding loaded, shape=(768,)
  [Load] Chunk 3 embedding loaded, shape=(768,)
  [Load] Chunk 4 embedding loaded, shape=(768,)
  [Load] Chunk 5 embedding loaded, shape=(768,)
  [Load] Chunk 6 embedding loaded, shape=(768,)
  [Load] Chunk 7 embedding loaded, shape=(768,)
  [Load] Chunk 8 embedding loaded, shape=(768,)
  [Load] Chunk 9 embedding loaded, shape=(768,)
  [Load] Chunk 10 embedding loaded, shape=(768,)
  [Load] Chunk 11 embedding loaded, shape=(768,)
  [Load] Chunk 12 embedding loaded, shape=(768,)
  [Load] Chunk 13 embedding loaded, shape=(768,)
  [Load] Chunk 14 embedding loaded

C:\Users\jagad\anaconda3\envs\testingopenai\Lib\site-packages\vertexai\_model_garden\_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


[Query] Query embedding vector size: 768
[Compare] Computing cosine similarities...
  [Compare] Chunk 0: similarity=0.5737
  [Compare] Chunk 1: similarity=0.5842
  [Compare] Chunk 2: similarity=0.4889
  [Compare] Chunk 3: similarity=0.5129
  [Compare] Chunk 4: similarity=0.5345
  [Compare] Chunk 5: similarity=0.4514
  [Compare] Chunk 6: similarity=0.4563
  [Compare] Chunk 7: similarity=0.5687
  [Compare] Chunk 8: similarity=0.5463
  [Compare] Chunk 9: similarity=0.5511
  [Compare] Chunk 10: similarity=0.5008
  [Compare] Chunk 11: similarity=0.4684
  [Compare] Chunk 12: similarity=0.4830
  [Compare] Chunk 13: similarity=0.4997
  [Compare] Chunk 14: similarity=0.5133
  [Compare] Chunk 15: similarity=0.5782
  [Compare] Chunk 16: similarity=0.5957
  [Compare] Chunk 17: similarity=0.6118
  [Compare] Chunk 18: similarity=0.5548
  [Compare] Chunk 19: similarity=0.5359
  [Compare] Chunk 20: similarity=0.5087
  [Compare] Chunk 21: similarity=0.5448
  [Compare] Chunk 22: similarity=0.5361
  [Com

C:\Users\jagad\anaconda3\envs\testingopenai\Lib\site-packages\vertexai\generative_models\_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


[Gemini] Response received (length: 1551)

=== Explanation ===
Of course! Here is that explanation broken down in simple terms for a student.

***

Imagine your body has its own super-fast internet or messaging system. This is your **nervous system**.

Here’s what the text explains about it:

*   **Special Messenger Cells:** While all cells in your body can react to things happening around them (a "stimulus," like heat or touch), the cells of your nervous system are experts. Their main job is to act like messengers that carry signals very, very quickly across your body.

*   **Where You Find Them:** These special cells make up your body's "command center" and "wiring": the **brain**, the **spinal cord**, and all the **nerves**.

*   **The Name of the Cell:** Each one of these special messenger cells is called a **neuron**.

### The Parts of a Neuron

Think of a neuron like a tree with one very long root.

1.  **The Cell Body:** This is the main part of the cell, like the base of the tr

In [44]:
import numpy as np
import pandas as pd
from io import BytesIO
from google.cloud import storage
from vertexai.preview.language_models import TextEmbeddingModel
from vertexai.preview.generative_models import GenerativeModel
from numpy.linalg import norm

PROJECT_ID = "agentic-ai-project-466814"
BUCKET_NAME = "studyplanandcontent"
GCS_META_PATH = "embeddingsoutput/iesc106_chunk_metadata.csv"

def preview_full_sentences_simple(text, max_chars=500):
    """
    Returns text truncated at the last period before max_chars to avoid incomplete sentences.
    If no period found, returns raw truncated snippet.
    """
    if len(text) <= max_chars:
        return text
    snippet = text[:max_chars]
    last_period = snippet.rfind('.')
    if last_period == -1:
        return snippet.strip()
    return snippet[:last_period + 1].strip()

def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

print("[Init] Initializing Google Cloud Storage client...")
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)
print(f"[Init] Connected to bucket '{BUCKET_NAME}'")

print(f"[Load] Downloading metadata CSV from gs://{BUCKET_NAME}/{GCS_META_PATH} ...")
meta_blob = bucket.blob(GCS_META_PATH)
csv_buf = BytesIO()
meta_blob.download_to_file(csv_buf)
csv_buf.seek(0)
meta_df = pd.read_csv(csv_buf)
num_chunks = len(meta_df)
print(f"[Load] Loaded metadata for {num_chunks} chunks")

print("[Load] Downloading all chunk embeddings...")
embeddings = []
for idx, row in meta_df.iterrows():
    emb_blob = bucket.blob(row['embedding_path'])
    emb_buf = BytesIO()
    emb_blob.download_to_file(emb_buf)
    emb_buf.seek(0)
    vector = np.load(emb_buf)
    embeddings.append(vector)
    print(f"  [Load] Chunk {idx} embedding loaded, shape={vector.shape}")

embeddings = np.array(embeddings)
print(f"[Load] All embeddings loaded with shape {embeddings.shape}")

# User query to search
user_question = "What is nervous tissue?"
print(f"[Query] Embedding user question: '{user_question}'")
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")
query_vec = embedding_model.get_embeddings([user_question])[0].values
print(f"[Query] Query embedding vector size: {len(query_vec)}")

print("[Compare] Computing cosine similarities for each chunk...")
similarities = []
for idx, chunk_vec in enumerate(embeddings):
    sim = cosine_similarity(query_vec, chunk_vec)
    similarities.append(sim)
    print(f"  [Compare] Chunk {idx}: similarity={sim:.4f}")

best_idx = int(np.argmax(similarities))
best_similarity = similarities[best_idx]
best_chunk = meta_df.iloc[best_idx]

print(f"\n[Result] Best chunk index: {best_idx} with similarity: {best_similarity:.4f}")

# Generate preview of chunk text ending at a full sentence
preview_text = preview_full_sentences_simple(best_chunk['text'], max_chars=500)
print(f"[Result] Chunk text preview (full sentences):\n{preview_text}\n")

# Generate explanation with Gemini
print("[Gemini] Initializing Gemini generative model...")
gemini = GenerativeModel("gemini-2.5-pro")

prompt = f'''
Here's a quote from the source document:

"{best_chunk["text"]}"

Explain the above in simple terms for a student.
'''

print("[Gemini] Sending prompt to Gemini model...")
response = gemini.generate_content(prompt)
print(f"[Gemini] Response received (length: {len(response.text)})")

print("\n=== Explanation ===")
print(response.text)
print("=" * 80)


[Init] Initializing Google Cloud Storage client...
[Init] Connected to bucket 'studyplanandcontent'
[Load] Downloading metadata CSV from gs://studyplanandcontent/embeddingsoutput/iesc106_chunk_metadata.csv ...
[Load] Loaded metadata for 31 chunks
[Load] Downloading all chunk embeddings...
  [Load] Chunk 0 embedding loaded, shape=(768,)
  [Load] Chunk 1 embedding loaded, shape=(768,)
  [Load] Chunk 2 embedding loaded, shape=(768,)
  [Load] Chunk 3 embedding loaded, shape=(768,)
  [Load] Chunk 4 embedding loaded, shape=(768,)
  [Load] Chunk 5 embedding loaded, shape=(768,)
  [Load] Chunk 6 embedding loaded, shape=(768,)
  [Load] Chunk 7 embedding loaded, shape=(768,)
  [Load] Chunk 8 embedding loaded, shape=(768,)
  [Load] Chunk 9 embedding loaded, shape=(768,)
  [Load] Chunk 10 embedding loaded, shape=(768,)
  [Load] Chunk 11 embedding loaded, shape=(768,)
  [Load] Chunk 12 embedding loaded, shape=(768,)
  [Load] Chunk 13 embedding loaded, shape=(768,)
  [Load] Chunk 14 embedding loaded

C:\Users\jagad\anaconda3\envs\testingopenai\Lib\site-packages\vertexai\_model_garden\_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


[Query] Query embedding vector size: 768
[Compare] Computing cosine similarities for each chunk...
  [Compare] Chunk 0: similarity=0.5609
  [Compare] Chunk 1: similarity=0.5693
  [Compare] Chunk 2: similarity=0.4962
  [Compare] Chunk 3: similarity=0.5130
  [Compare] Chunk 4: similarity=0.5360
  [Compare] Chunk 5: similarity=0.4600
  [Compare] Chunk 6: similarity=0.4740
  [Compare] Chunk 7: similarity=0.5611
  [Compare] Chunk 8: similarity=0.5485
  [Compare] Chunk 9: similarity=0.5467
  [Compare] Chunk 10: similarity=0.5018
  [Compare] Chunk 11: similarity=0.4634
  [Compare] Chunk 12: similarity=0.4756
  [Compare] Chunk 13: similarity=0.5027
  [Compare] Chunk 14: similarity=0.5177
  [Compare] Chunk 15: similarity=0.5711
  [Compare] Chunk 16: similarity=0.5933
  [Compare] Chunk 17: similarity=0.6047
  [Compare] Chunk 18: similarity=0.5420
  [Compare] Chunk 19: similarity=0.5226
  [Compare] Chunk 20: similarity=0.4970
  [Compare] Chunk 21: similarity=0.5296
  [Compare] Chunk 22: similarit

C:\Users\jagad\anaconda3\envs\testingopenai\Lib\site-packages\vertexai\generative_models\_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


[Gemini] Response received (length: 1961)

=== Explanation ===
Of course! Here is that explanation broken down into simple terms for a student.

***

Imagine your body is a big, busy city. For anything to happen, different parts of the city need to talk to each other, and they need to do it fast!

### Your Body's Super-Fast Messengers

The quote explains that your body has a special, high-speed communication system.

*   **All cells can react:** Think of every cell in your body as a person. Every person can react to things, like moving away if something is too hot.
*   **Nerve cells are specialists:** But some cells, called **nerve cells** (or **neurons**), are like professional messengers. Their only job is to receive a message (a "stimulus") and pass it along to another part of the body at incredible speed.

### Where Are These Messengers Found?

These special nerve cells make up your body's main control network:

*   **The Brain:** The main control center.
*   **The Spinal Cord:** T

In [46]:
from vertexai.preview.generative_models import GenerativeModel

# Initialize Gemini model
gemini = GenerativeModel("gemini-2.5-pro")

def create_study_plan(subject_name, total_days=14, session_minutes=60):
    prompt = f"""
Act as an expert school tutor who is planning a self-study schedule for a middle school student.
Do NOT quote or refer to the book text. 
Assume the subject is "{subject_name}". Assume the kid has access to good notes, book material, and practice questions.
Create a detailed, step-by-step study plan for {total_days} days, with each study session lasting {session_minutes} minutes (1 hour per day).
Split the subject into logical topics and assign each to different days as appropriate.
For each day, provide:
- Topic(s) to be studied
- 3-5 bullet points, in your own words, summarizing what the student should cover or focus on
- The type of practice or revision the student should do
- Anything to pay special attention to or tips

Be concise, clear, and motivating.
"""

    print("[Gemini] Requesting automated study plan from Gemini...")
    response = gemini.generate_content(prompt)
    print("[Gemini] Study plan generated.\n")
    print(response.text)

# Example usage (update subject_name as needed)
create_study_plan(subject_name="Tissues in Science (Grade 9 NCERT)", total_days=14, session_minutes=60)


[Gemini] Requesting automated study plan from Gemini...
[Gemini] Study plan generated.

Of course! Here is a detailed, 14-day self-study plan for the chapter "Tissues." Let's get you prepared and confident. You've got this!

### **Your 14-Day Mission: Master the World of Tissues**

This plan is designed for one hour of focused study each day. Remember to find a quiet spot, keep your phone away, and have your notes and practice materials ready. Let's begin!

---

#### **PART 1: PLANT TISSUES (Days 1-4)**

**Day 1: The Basics & Growing Tissues (Meristematic)**
*   **Topic:** Introduction to Tissues & Meristematic Tissue.
*   **What to Cover:**
    *   Understand what a tissue is and why complex organisms like plants and animals need them for organized work.
    *   Focus on what makes meristematic tissues special: they are actively dividing cells responsible for growth.
    *   Learn the three types based on their location: Apical (at the tips for length), Lateral (on the sides for width